In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import requests
import numpy as np
import time
# Specify the path to ChromeDriver
chrome_driver_path ='chromedriver-mac-x64/chromedriver'
CommunityNotes=pd.read_csv("CommunityNotes.csv")


In [26]:
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)


In [23]:
#target_weibos=pd.DataFrame(columns=['target','user_id','user_name','post_time','post','note_author','note_type','note_content'])

target_weibos=pd.read_csv("TargetWeibos.csv")

In [24]:
CommunityNotes_targets=list(set(CommunityNotes.target.unique()).difference(set(target_weibos.target.unique())))

In [28]:
len(CommunityNotes_targets)

46

In [17]:
CommunityNotes_targets.index('http://t.cn/A6HXoEkn')

3015

In [27]:
for i in range(len(CommunityNotes_targets)):
    if CommunityNotes_targets[i].startswith('http://t.cn'):
        driver.get(CommunityNotes_targets[i])
        time.sleep(10)
        print(CommunityNotes_targets[i])
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        user=soup.find('a',{'class':'ALink_default_2ibt1 head_cut_2Zcft head_name_24eEB'})
        if user:
            user_id=user.get('usercard')
            user_name=user.find('span').get_text()
            post=soup.find('div',{'class':'detail_wbtext_4CRf9'})
            post_time=soup.find('a',{'class':'head-info_time_6sFQg'}).get_text(strip=True)
            notes=soup.find_all('div',{'class':'community-note-detail_item_2kRdt'})
            if len(notes)!=0:
                for note in notes:
                    note_author=note.find('div',{'class':'community-note-detail_t11_14-ls community-note-detail_pointer_3Ro0s'}).get_text(strip=True)
                    note_type=note.find('div',{'class':'community-note-detail_t12_2O51m'}).get_text(strip=True)
                    note_content=note.find('div',{'class':'community-note-detail_t2_NIKuE community-note-detail_es_1U-kA community-note-detail_pointer_3Ro0s'}).get_text(strip=True)
                    new_row = {'target':CommunityNotes_targets[i],'user_id':user_id,'user_name':user_name,'post_time':post_time,'post':post,'note_author':note_author,'note_type':note_type,'note_content':note_content}
                    target_weibos = target_weibos.append(new_row, ignore_index=True)
            else:
                new_row = {'target':CommunityNotes_targets[i],'user_id':user_id,'user_name':user_name,'post_time':post_time,'post':post,'note_author':None,'note_type':None,'note_content':None}
                target_weibos = target_weibos.append(new_row, ignore_index=True)

        else:
            new_row = {'target':CommunityNotes_targets[i],'user_id':None,'user_name':None,'post_time':None,'post':None,'note_author':None,'note_type':None,'note_content':None}
            target_weibos = target_weibos.append(new_row, ignore_index=True)
        target_weibos.to_csv("TargetWeibos.csv",index=False)
